In [1]:
from pydantic import BaseModel, Field

class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")

Notice how we take care to add a **description to each field** in the schema.

This is important because:

- The **field name** and **description** are the primary clues the LLM uses  
  to determine what part of the output should be placed in each field.
- Well-written descriptions improve the **accuracy and consistency** of the structured output.

---

Alternatively, instead of using a class-based or model-based schema,  
we can define the schema directly using **raw JSONSchema notation**.

Here’s what that might look like:

```JSON
{'properties': {'setup': {'description': 'The setup of the joke',
    'title': 'Setup',
    'type': 'string'},
 'punchline': {'description': 'The punchline to the joke',
    'title': 'Punchline',
    'type': 'string'}},
 'required': ['setup', 'punchline'],
 'title': 'Joke',
 'type': 'object'}
```

Let's use this schema:

In [2]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(model="gpt-4o", temperature=0, azure_deployment="gpt-4o", api_version="2024-10-21") 
model = model.with_structured_output(Joke)

model.invoke("Tell me a joke about cats")

Joke(setup='Why did the cat sit on the computer?', punchline='To keep an eye on the mouse!')

A couple of things to notice when working with structured output:

- We start by **creating the model instance** as usual, specifying:
  - The **model name**
  - Other parameters such as temperature

- A **low temperature** is typically recommended for structured output.  
  This reduces the likelihood of the LLM generating invalid or unpredictable output  
  that fails to conform to the schema.

- We then **attach the schema** to the model using `.with_structured_output`.  
  This returns a new model object that is configured to:
  - Produce output matching the schema
  - Perform **validation** using the provided schema object (e.g., Pydantic or Zod)

- If the LLM’s output **does not match the schema**, a **validation error** will be raised  
  instead of silently returning incorrect data.

- Finally, we **invoke the model** with our free-form input  
  and receive a **structured response** that conforms to the schema.

This approach helps ensure that outputs are **reliable**, **machine-readable**, and **safe to use in downstream applications**.

Let's see the [next approach](../README.md#intermediate-output)